# Sistema de Detecção de Fadiga

Sistema que detecta fadiga usando webcam e modelo treinado.

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import time
import sys
from collections import deque
from pathlib import Path

# Carrega o modelo treinado
sys.path.append('modelos_xgb')
from pipeline import PipelineFadiga

print("Bibliotecas carregadas")

In [ ]:
# Inicializa MediaPipe para detecção facial
mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

print("MediaPipe pronto")

In [ ]:
# Carrega o modelo XGBoost treinado
modelo_dir = "modelos_xgb"

if not Path(modelo_dir).exists():
    print("Erro: Execute primeiro o notebook de treino")
    raise FileNotFoundError(f"Pasta não encontrada: {modelo_dir}")

pipeline = PipelineFadiga(modelo_dir)
print(f"Modelo carregado - Precisão: {pipeline.class_info['accuracy']:.1%}")

In [ ]:
# Detector de Fadiga com Calibração Personalizada
# Sistema que aprende os padrões de cada usuário

import json
from pathlib import Path

class DetectorFadiga:
    def __init__(self, pipeline, usuario="default"):
        self.pipeline = pipeline
        self.usuario = usuario
        
        # Buffers para dados
        self.buffer = deque(maxlen=90)
        self.ear_history = deque(maxlen=200)
        self.mar_history = deque(maxlen=200)
        self.piscadas = []
        
        # Pontos dos olhos e boca
        self.olho_direito = [33, 160, 158, 133, 153, 144]
        self.olho_esquerdo = [362, 385, 387, 263, 373, 380]
        self.boca = [13, 14, 78, 308, 0, 17]
        
        # Configurações do usuário
        self.config = {
            'ear_normal': 0.30,
            'ear_limite': 0.25,
            'mar_normal': 0.60,
            'blink_normal': 18.0,
            'calibrado': False,
            'sessoes': 0
        }
        
        # Estados atuais
        self.ear_atual = 0.0
        self.mar_atual = 0.0
        self.blink_rate = 0.0
        self.predicao = "Alerta"
        self.confianca = 0.0
        
        self.carregar_config()
    
    def salvar_config(self):
        try:
            pasta = Path("perfis_usuarios")
            pasta.mkdir(exist_ok=True)
            
            arquivo = pasta / f"{self.usuario}.json"
            with open(arquivo, 'w') as f:
                json.dump(self.config, f, indent=2)
        except Exception as e:
            print(f"Erro ao salvar: {e}")
    
    def carregar_config(self):
        try:
            arquivo = Path("perfis_usuarios") / f"{self.usuario}.json"
            if arquivo.exists():
                with open(arquivo, 'r') as f:
                    config_salva = json.load(f)
                    self.config.update(config_salva)
                print(f"Perfil carregado: {self.config['sessoes']} sessões")
        except Exception as e:
            print(f"Erro ao carregar: {e}")
    
    def calibrar(self, cap, tempo=20):
        print(f"Calibração ({tempo}s)")
        print("Fique natural e pisque normalmente")
        
        ear_dados = []
        mar_dados = []
        piscadas_cal = []
        
        inicio = time.time()
        frames = 0
        ear_historico = []
        
        while time.time() - inicio < tempo:
            ret, frame = cap.read()
            if not ret:
                continue
                
            frame = cv2.flip(frame, 1)
            h, w = frame.shape[:2]
            
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb_frame)
            
            if results.multi_face_landmarks:
                landmarks = results.multi_face_landmarks[0].landmark
                
                ear_d = self.calcular_ear(self.olho_direito, landmarks, w, h)
                ear_e = self.calcular_ear(self.olho_esquerdo, landmarks, w, h)
                ear = (ear_d + ear_e) / 2.0
                
                mar = self.calcular_mar(landmarks, w, h)
                
                if ear > 0.1 and mar > 0:
                    ear_dados.append(ear)
                    mar_dados.append(mar)
                    ear_historico.append(ear)
                    
                    # Detectar piscadas durante calibração
                    if len(ear_historico) >= 10:
                        tendencia = np.diff(ear_historico[-10:])
                        
                        for i in range(len(tendencia) - 3):
                            if (tendencia[i] < -0.04 and 
                                tendencia[i+1] < 0 and 
                                tendencia[i+2] > 0.04):
                                
                                min_ear = min(ear_historico[i:i+3])
                                if min_ear < 0.3:
                                    piscadas_cal.append(min_ear)
            
            # Interface de calibração
            progresso = (time.time() - inicio) / tempo
            bar_w = int(w * 0.7)
            bar_x = (w - bar_w) // 2
            bar_y = h // 2
            
            cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_w, bar_y + 25), (40, 40, 40), -1)
            cv2.rectangle(frame, (bar_x, bar_y), (bar_x + int(bar_w * progresso), bar_y + 25), (0, 200, 0), -1)
            
            tempo_restante = tempo - (time.time() - inicio)
            cv2.putText(frame, f"Calibracao: {tempo_restante:.0f}s", 
                       (bar_x, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            if len(ear_dados) > 0:
                cv2.putText(frame, f"EAR: {ear_dados[-1]:.3f}", (20, 40), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, f"Piscadas: {len(piscadas_cal)}", (20, 70), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            
            cv2.imshow('Calibracao', frame)
            
            if cv2.waitKey(1) & 0xFF == 27:
                return False
            
            frames += 1
        
        print(f"Dados: {len(ear_dados)} EAR, {len(piscadas_cal)} piscadas")
        
        if len(ear_dados) < 20:
            print("Poucas amostras")
            return False
        
        # Processar dados
        ear_limpo = self.limpar_dados(ear_dados)
        mar_limpo = self.limpar_dados(mar_dados)
        
        if len(ear_limpo) < 10:
            print("Dados com muito ruído")
            return False
        
        # Calcular configurações personalizadas
        ear_normal = np.percentile(ear_limpo, 60)
        mar_normal = np.percentile(mar_limpo, 50)
        ear_std = np.std(ear_limpo)
        
        self.config['ear_normal'] = ear_normal
        self.config['ear_limite'] = max(0.15, ear_normal - (2.0 * ear_std))
        self.config['mar_normal'] = mar_normal
        
        # Blink rate
        tempo_total = frames / 30.0
        if len(piscadas_cal) > 0:
            blink_baseline = (len(piscadas_cal) / tempo_total) * 60.0
        else:
            blink_baseline = 15.0
        
        self.config['blink_normal'] = max(8.0, min(35.0, blink_baseline))
        self.config['calibrado'] = True
        self.config['sessoes'] += 1
        
        self.salvar_config()
        
        print(f"Calibração concluída!")
        print(f"EAR: {ear_normal:.3f} (limite: {self.config['ear_limite']:.3f})")
        print(f"Blink rate: {blink_baseline:.1f} bpm")
        
        cv2.destroyWindow('Calibracao')
        return True
    
    def limpar_dados(self, dados):
        dados = np.array(dados)
        q1, q3 = np.percentile(dados, [25, 75])
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        return dados[(dados >= lower) & (dados <= upper)]
    
    def distancia(self, p1, p2):
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
    
    def calcular_ear(self, pontos_olho, landmarks, w, h):
        try:
            pontos = []
            for idx in pontos_olho:
                x = landmarks[idx].x * w
                y = landmarks[idx].y * h
                pontos.append([x, y])
            
            if len(pontos) >= 6:
                d1 = self.distancia(pontos[1], pontos[5])
                d2 = self.distancia(pontos[2], pontos[4])
                d3 = self.distancia(pontos[0], pontos[3])
                
                if d3 > 0:
                    ear = (d1 + d2) / (2.0 * d3)
                    return max(0.0, min(1.0, ear))
            
            return 0.0
        except:
            return 0.0
    
    def calcular_mar(self, landmarks, w, h):
        try:
            pontos = []
            for idx in self.boca:
                x = landmarks[idx].x * w
                y = landmarks[idx].y * h
                pontos.append([x, y])
            
            if len(pontos) >= 6:
                d_vert1 = self.distancia(pontos[0], pontos[1])
                d_vert2 = self.distancia(pontos[4], pontos[5])
                d_horiz = self.distancia(pontos[2], pontos[3])
                
                if d_horiz > 0:
                    mar = (d_vert1 + d_vert2 * 0.5) / d_horiz
                    return max(0.0, min(3.0, mar))
            
            return 0.0
        except:
            return 0.0
    
    def detectar_piscada(self, ear):
        if len(self.ear_history) >= 10:
            historico = list(self.ear_history)[-10:]
            historico.append(ear)
            
            tendencia = np.diff(historico)
            limite_flex = self.config['ear_limite'] * 1.3
            
            for i in range(len(tendencia) - 4):
                queda = tendencia[i] < -0.025
                baixo = tendencia[i+1] < 0.015
                subida = tendencia[i+2] > 0.015 or tendencia[i+3] > 0.015
                
                if queda and subida:
                    min_ear = min(historico[i:i+4])
                    max_ear = max(historico[i:i+4])
                    
                    if (min_ear < limite_flex and 
                        max_ear > min_ear + 0.03 and 
                        max_ear < 0.6):
                        
                        agora = time.time()
                        if not self.piscadas or (agora - self.piscadas[-1]['tempo']) > 0.15:
                            self.piscadas.append({'tempo': agora, 'ear_min': min_ear})
                            return True
        
        return False
    
    def calcular_blink_rate(self):
        agora = time.time()
        janela = 30.0
        
        piscadas_recentes = [p for p in self.piscadas 
                           if agora - p['tempo'] <= janela]
        
        if len(piscadas_recentes) >= 2:
            tempo_real = min(janela, agora - self.piscadas[0]['tempo'])
            rate = (len(piscadas_recentes) / tempo_real) * 60.0
            return min(60.0, max(0.0, rate))
        
        return 0.0
    
    def processar_features(self, landmarks, w, h):
        ear_d = self.calcular_ear(self.olho_direito, landmarks, w, h)
        ear_e = self.calcular_ear(self.olho_esquerdo, landmarks, w, h)
        ear = (ear_d + ear_e) / 2.0
        
        mar = self.calcular_mar(landmarks, w, h)
        
        self.ear_history.append(ear)
        self.mar_history.append(mar)
        
        self.detectar_piscada(ear)
        
        # PERCLOS
        if len(self.ear_history) >= 60:
            window = list(self.ear_history)[-60:]
            frames_fechados = sum(1 for e in window if e < self.config['ear_limite'])
            perclos = (frames_fechados / len(window)) * 100
        else:
            perclos = 0.0
        
        blink_rate = self.calcular_blink_rate()
        
        # Head stability
        try:
            nose1 = landmarks[1]
            nose2 = landmarks[9]
            head_stability = abs(nose1.x - nose2.x) + abs(nose1.y - nose2.y)
        except:
            head_stability = 0.0
        
        self.ear_atual = ear
        self.mar_atual = mar
        self.blink_rate = blink_rate
        
        return [perclos, mar, blink_rate, head_stability]
    
    def processar_frame(self, frame):
        h, w = frame.shape[:2]
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)
        
        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0].landmark
            features = self.processar_features(landmarks, w, h)
            self.buffer.append(features)
            return True
        return False
    
    def predizer(self):
        if len(self.buffer) == 90:
            sequence = np.array(list(self.buffer))
            pred, probs, classe = self.pipeline.predict_sequence(sequence)
            
            self.predicao = classe
            self.confianca = max(probs.values())
            
            return True
        return False
    
    def desenhar_interface(self, frame):
        h, w = frame.shape[:2]
        
        # Painel principal
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (550, 300), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
        
        # Status
        cor = (0, 255, 0) if self.predicao == "Alerta" else (0, 0, 255)
        cv2.putText(frame, f"Status: {self.predicao}", (20, 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, cor, 2)
        
        cv2.putText(frame, f"Confianca: {self.confianca:.1%}", (20, 75), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.putText(frame, f"Buffer: {len(self.buffer)}/90", (20, 105), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Perfil
        if self.config['calibrado']:
            cv2.putText(frame, f"Perfil: Ativo (sessao {self.config['sessoes']})", 
                       (20, 135), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        else:
            cv2.putText(frame, "Perfil: Nao calibrado", (20, 135), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        
        # Features
        cv2.putText(frame, "=== METRICAS ===", (300, 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)
        
        cv2.putText(frame, f"EAR: {self.ear_atual:.3f}", (300, 70), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(frame, f"MAR: {self.mar_atual:.3f}", (300, 95), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Blink rate com cor
        cor_blink = (0, 255, 0) if self.blink_rate > 0 else (0, 0, 255)
        cv2.putText(frame, f"Blink Rate: {self.blink_rate:.1f}", (300, 120), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, cor_blink, 1)
        
        cv2.putText(frame, f"Piscadas: {len(self.piscadas)}", (300, 145), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Barra EAR
        cv2.putText(frame, "EAR vs Limite:", (20, 180), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        bar_w = 200
        bar_x = 20
        bar_y = 190
        
        cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_w, bar_y + 15), (50, 50, 50), -1)
        
        # Limite
        limite_pos = int((self.config['ear_limite'] / 0.5) * bar_w)
        cv2.line(frame, (bar_x + limite_pos, bar_y), (bar_x + limite_pos, bar_y + 15), (0, 255, 255), 2)
        
        # EAR atual
        ear_pos = int((self.ear_atual / 0.5) * bar_w)
        ear_pos = min(bar_w, max(0, ear_pos))
        cor_ear = (0, 255, 0) if self.ear_atual > self.config['ear_limite'] else (0, 0, 255)
        cv2.circle(frame, (bar_x + ear_pos, bar_y + 7), 5, cor_ear, -1)
        
        return frame

# Criar detector
detector = DetectorFadiga(pipeline, usuario="usuario1")
print("Detector criado!")
print(f"Status: {'Calibrado' if detector.config['calibrado'] else 'Novo usuário'}")
if detector.config['calibrado']:
    print(f"EAR limite: {detector.config['ear_limite']:.3f}")
    print(f"Sessões: {detector.config['sessoes']}")

In [ ]:
def testar_sistema():
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Erro: Não conseguiu abrir a câmera")
        return
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    print("Câmera inicializada")
    
    # Calibração
    if detector.config['calibrado']:
        print(f"Perfil encontrado (sessão {detector.config['sessoes']})")
        print(f"EAR limite: {detector.config['ear_limite']:.3f}")
        
        resposta = input("Recalibrar? (s/N): ").lower()
        if resposta in ['s', 'sim']:
            sucesso = detector.calibrar(cap, 20)
        else:
            sucesso = True
            print("Usando perfil existente")
    else:
        print("Novo usuário - fazendo calibração")
        sucesso = detector.calibrar(cap, 25)
    
    if not sucesso:
        print("Falha na calibração")
        cap.release()
        return
    
    # Detecção
    print("\nIniciando detecção em tempo real")
    print("Controles: q=Sair, r=Recalibrar, p=Salvar")
    
    cv2.namedWindow('Detecção de Fadiga', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Detecção de Fadiga', 800, 600)
    
    contador = 0
    inicio = time.time()
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                continue
            
            frame = cv2.flip(frame, 1)
            
            # Processar frame
            face_ok = detector.processar_frame(frame)
            predicao_ok = detector.predizer()
            
            # Interface
            frame = detector.desenhar_interface(frame)
            
            # Status da face
            h, w = frame.shape[:2]
            status_face = "Face OK" if face_ok else "Sem face"
            cor_face = (0, 255, 0) if face_ok else (0, 0, 255)
            cv2.putText(frame, status_face, (w-120, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, cor_face, 2)
            
            # Controles
            cv2.putText(frame, "q=Sair | r=Recalibrar | p=Salvar", (10, h-20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            cv2.imshow('Detecção de Fadiga', frame)
            
            # Estatísticas a cada 30 frames
            contador += 1
            if contador % 30 == 0:
                tempo_decorrido = time.time() - inicio
                fps = contador / tempo_decorrido
                
                print(f"FPS: {fps:.1f} | Buffer: {len(detector.buffer)}/90 | "
                      f"Piscadas: {len(detector.piscadas)} | Status: {detector.predicao}")
            
            # Teclas
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q') or key == 27:
                break
            
            elif key == ord('r'):
                print("Recalibrando...")
                detector.buffer.clear()
                detector.ear_history.clear()
                detector.piscadas.clear()
                
                cv2.destroyWindow('Detecção de Fadiga')
                
                sucesso = detector.calibrar(cap, 20)
                if sucesso:
                    print("Recalibração concluída")
                else:
                    print("Recalibração falhou")
                
                cv2.namedWindow('Detecção de Fadiga', cv2.WINDOW_NORMAL)
                cv2.resizeWindow('Detecção de Fadiga', 800, 600)
            
            elif key == ord('p'):
                detector.salvar_config()
                print("Perfil salvo")
            
            # Verificar se janela foi fechada
            if cv2.getWindowProperty('Detecção de Fadiga', cv2.WND_PROP_VISIBLE) < 1:
                break
    
    except KeyboardInterrupt:
        print("Interrompido pelo usuário")
    
    except Exception as e:
        print(f"Erro: {e}")
    
    finally:
        cap.release()
        cv2.destroyAllWindows()
        
        # Relatório final
        if contador > 0:
            tempo_total = time.time() - inicio
            
            print(f"\nRelatório:")
            print(f"Tempo: {tempo_total:.1f}s")
            print(f"Frames: {contador}")
            print(f"FPS médio: {contador/tempo_total:.1f}")
            print(f"Piscadas detectadas: {len(detector.piscadas)}")
            
            if detector.config['calibrado']:
                print(f"EAR limite: {detector.config['ear_limite']:.3f}")
        
        detector.salvar_config()
        print("Sistema encerrado")

print("Sistema pronto!")

In [ ]:
# Teste Rápido
# Execute apenas se houver problemas com a célula principal

print("=== Teste Básico ===")
print("Verificando câmera...")

def verificar_camera():
    try:
        cap = cv2.VideoCapture(0)
        if cap.isOpened():
            cap.release()
            return True
        return False
    except:
        return False

if not verificar_camera():
    print("❌ Câmera não disponível")
    print("Soluções:")
    print("- Feche outros apps usando a câmera")
    print("- Reinicie o kernel")
else:
    print("✅ Câmera OK")
    print("Use a célula principal (Sistema de Detecção)")

In [ ]:
# Teste do Sistema de Detecção de Fadiga

print("=== Detecção de Fadiga ===")
print("Sistema com calibração personalizada")

# Verificar se detector existe
if 'detector' not in globals():
    print("Execute as células anteriores primeiro")
else:
    print(f"Usuário: {detector.usuario}")
    print(f"Status: {'Calibrado' if detector.config['calibrado'] else 'Novo'}")
    
    if detector.config['calibrado']:
        print(f"Sessões: {detector.config['sessoes']}")
        print(f"EAR limite: {detector.config['ear_limite']:.3f}")
    
    print("\nPressione Ctrl+C para parar")
    
    try:
        # Garantir limpeza
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        
        testar_sistema()
        
    except Exception as e:
        print(f"Erro: {e}")
        cv2.destroyAllWindows()

In [ ]:
# Resumo do Sistema

print("Sistema de Detecção de Fadiga")
print("=============================")
print("Funcionalidades:")
print("- Calibração personalizada por usuário")
print("- Detecção de piscadas em tempo real")
print("- Cálculo de EAR, MAR e PERCLOS")
print("- Perfis salvos automaticamente")
print("- Interface visual com métricas")
print("")
print("Como usar:")
print("1. Execute a célula anterior")
print("2. Faça a calibração (20-25s)")
print("3. Use normalmente")
print("")
print("Controles:")
print("q = Sair")
print("r = Recalibrar")
print("p = Salvar perfil")